# LangChain、Deep Lake、OpenAIを使った質問応答システム

このノートブックでは、LangChain、ベクトルストアとしての[Deep Lake](https://activeloop.ai/)、OpenAI埋め込みを使用して質問応答システムを実装する方法を示します。これを実現するために以下のステップを実行します：

1. Deep Lakeテキストデータセットを読み込む
2. [LangChainでDeep Lakeベクトルストア](https://docs.activeloop.ai/tutorials/vector-store/deep-lake-vector-store-in-langchain)を初期化する
3. ベクトルストアにテキストを追加する
4. データベースでクエリを実行する
5. 完了！

また、あらゆる種類のデータ（PDF、json、csv、テキスト）に対する質問応答など、他のチュートリアルもフォローできます：Deep Lakeに保存された[あらゆるデータとのチャット](https://www.activeloop.ai/resources/data-chad-an-ai-app-with-lang-chain-deep-lake-to-chat-with-any-data/)、[コード理解](https://www.activeloop.ai/resources/lang-chain-gpt-4-for-code-understanding-twitter-algorithm/)、[PDFに対する質問応答](https://www.activeloop.ai/resources/ultimate-guide-to-lang-chain-deep-lake-build-chat-gpt-to-answer-questions-on-your-financial-data/)、または[楽曲推薦](https://www.activeloop.ai/resources/3-ways-to-build-a-recommendation-engine-for-songs-with-lang-chain/)などがあります。

## 要件のインストール
以下のパッケージをインストールしましょう。

In [ ]:
!pip install deeplake langchain openai tiktoken

## 認証
OpenAI APIキーをここに入力してください：

In [2]:
import getpass
import os

os.environ['OPENAI_API_KEY'] = getpass.getpass()

··········


## Deep Lake テキストデータセットの読み込み
この例では、[cohere-wikipedia-22](https://app.activeloop.ai/davitbun/cohere-wikipedia-22) データセットの20,000サンプルのサブセットを使用します。

In [3]:
import deeplake

ds = deeplake.load("hub://activeloop/cohere-wikipedia-22-sample")
ds.summary()

\

Opening dataset in read-only mode as you don't have write permissions.


-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/cohere-wikipedia-22-sample



|

hub://activeloop/cohere-wikipedia-22-sample loaded successfully.

Dataset(path='hub://activeloop/cohere-wikipedia-22-sample', read_only=True, tensors=['ids', 'metadata', 'text'])

  tensor    htype     shape      dtype  compression
 -------   -------   -------    -------  ------- 
   ids      text    (20000, 1)    str     None   
 metadata   json    (20000, 1)    str     None   
   text     text    (20000, 1)    str     None   


いくつかのサンプルを見てみましょう：

In [4]:
ds[:3].text.data()["value"]

['The 24-hour clock is a way of telling the time in which the day runs from midnight to midnight and is divided into 24 hours, numbered from 0 to 23. It does not use a.m. or p.m. This system is also referred to (only in the US and the English speaking parts of Canada) as military time or (only in the United Kingdom and now very rarely) as continental time. In some parts of the world, it is called railway time. Also, the international standard notation of time (ISO 8601) is based on this format.',
 'A time in the 24-hour clock is written in the form hours:minutes (for example, 01:23), or hours:minutes:seconds (01:23:45). Numbers under 10 have a zero in front (called a leading zero); e.g. 09:07. Under the 24-hour clock system, the day begins at midnight, 00:00, and the last minute of the day begins at 23:59 and ends at 24:00, which is identical to 00:00 of the following day. 12:00 can only be mid-day. Midnight is called 24:00 and is used to mean the end of the day and 00:00 is used to me

## LangChainのDeep Lakeベクトルストア
`dataset_path`を定義しましょう。これは、Deep Lakeベクトルストアがテキスト埋め込みを格納する場所です。

In [5]:
dataset_path = 'wikipedia-embeddings-deeplake'

OpenAIの`text-embedding-3-small`を埋め込み関数として設定し、`dataset_path`でDeep Lakeベクトルストアを初期化します...

In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

embedding = OpenAIEmbeddings(model="text-embedding-3-small")
db = DeepLake(dataset_path, embedding=embedding, overwrite=True)

... そして、`add_texts`メソッドを使用して、一度に1バッチずつサンプルを追加して埋めていきます。

In [7]:
from tqdm.auto import tqdm

batch_size = 100

nsamples = 10  # for testing. Replace with len(ds) to append everything
for i in tqdm(range(0, nsamples, batch_size)):
    # find end of batch
    i_end = min(nsamples, i + batch_size)

    batch = ds[i:i_end]
    id_batch = batch.ids.data()["value"]
    text_batch = batch.text.data()["value"]
    meta_batch = batch.metadata.data()["value"]

    db.add_texts(text_batch, metadatas=meta_batch, ids=id_batch)

  0%|          | 0/1 [00:00<?, ?it/s]


creating embeddings: 100%|██████████| 1/1 [00:02<00:00,  2.11s/it]

100%|██████████| 10/10 [00:00<00:00, 462.42it/s]

Dataset(path='wikipedia-embeddings-deeplake', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (10, 1)      str     None   
 metadata     json      (10, 1)      str     None   
 embedding  embedding  (10, 1536)  float32   None   
    id        text      (10, 1)      str     None   


## データベースでユーザークエリを実行する
基盤となるDeep Lakeデータセットオブジェクトは`db.vectorstore.dataset`を通じてアクセス可能で、データ構造は`db.vectorstore.summary()`を使用して要約できます。これは10個のサンプルを持つ4つのテンソルを表示します：

In [8]:
db.vectorstore.summary()

Dataset(path='wikipedia-embeddings-deeplake', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (10, 1)      str     None   
 metadata     json      (10, 1)      str     None   
 embedding  embedding  (10, 1536)  float32   None   
    id        text      (10, 1)      str     None   


ベクトルストアにGPT-3.5-TurboをLLMとして使用したQAシステムをセットアップします。

In [9]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Re-load the vector store in case it's no longer initialized
# db = DeepLake(dataset_path = dataset_path, embedding_function=embedding)

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model='gpt-3.5-turbo'), chain_type="stuff", retriever=db.as_retriever())

プロンプトを実行して出力を確認してみましょう。内部的に、このAPIは埋め込み検索を実行して、LLMコンテキストに入力する最も関連性の高いデータを見つけます。

In [10]:
query = 'Why does the military not say 24:00?'
qa.run(query)

'The military prefers not to say 24:00 because they do not like to have two names for the same thing. Instead, they always say "23:59", which is one minute before midnight.'

以下のテキストを日本語に翻訳いたします：

これで完成です！

（「Et voila!」はフランス語の表現で、「さあ、できました！」「これで完成！」という意味の感嘆詞です。日本語では「これで完成です！」「はい、できあがり！」などと訳されます。）